In [1]:
project_list = [
    # 'assertj-assertions-generator',
    # 'commons-net',
    'commons-cli',
    'commons-csv',
    'commons-codec',
    'delight-nashorn-sandbox',
    'empire-db',
    'jimfs',
    'httpcore',
    'handlebars.java',
    'riptide',
    # 'commons-collections',
    # 'guava',
    # 'java-design-patterns',
    # 'jooby',
    # 'maven-dependency-plugin',
    # 'maven-shade-plugin',
    # 'sling-org-apache-sling-auth-core',
    # 'stream-lib'
]
seed_list = [
    # 0,
    # 42,
    # 123,
    # 216,
    # 1202,
    # 1999,
    # 2002,
    # 2024,
    # 31415,
    # 99999,
    'default',
    # 'fastest',
    # 'GC_test_order',
    # 'M_fewest_tests',
    # 'M_most_tests',
    # 'M_most_coverage',
    # 'M_most_similar',
    # 'M_most_different',
    # 'def_1_groups',
    # 'def_2_groups',
    # 'def_4_groups',
    # 'def_6_groups',
    # 'def_8_groups',
    # 'def_10_groups',
    # 'def_15_groups',
    # 'def_20_groups',
    # 'def_25_groups',
    # 'def_30_groups',
    # 'def_40_groups',
    # 'def_50_groups',
    # 'def_65_groups',
    # 'def_80_groups',
    # 'def_100_groups',
    'clz_clz-cvg_def',
    'clz_ln-cvg_def',
    'n-tst_clz-cvg_def',
    'n-tst_ln-cvg_def',
    '01-tst_clz-cvg_def',
    '01-tst_ln-cvg_def'
]
round_number = 6
seed_number = len(seed_list)
parsed_dir = 'controlled_parsed_data/both'
analyzed_dir = 'controlled_analyzed_data/both'

In [14]:
import json
import math
import pandas as pd
import numpy as np

cols = ['class'] + seed_list + ['mean_deviation']
def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as f:
        id_tuple_dict = json.load(f)
    with open(f'{file_path}/mutantId_runtimeList.json', 'r') as f:
        id_runtimes_dict = json.load(f)
    return id_tuple_dict, id_runtimes_dict


for p in project_list:
    df = pd.DataFrame(None, columns=cols)
    is_visited = False
    clz_runtimes_dict = dict()
    for s in seed_list:
        clz_ids_dict = dict()
        id_tuple_dict, id_runtimes_dict = get_info(f'controlled_parsed_data/both/{p}_{s}')
        with open(f'{analyzed_dir}/mutant_list/non-flaky/{p}_{s}.json', 'r') as f:
            non_flaky_list = json.load(f)
        if not is_visited:
            is_visited = True
            for mut_id in non_flaky_list:
                mut_tup = id_tuple_dict[mut_id]
                clz = mut_tup[0]
                clz_runtimes_dict[clz] = []
        for mut_id in non_flaky_list:
            mut_tup = id_tuple_dict[mut_id]
            clz = mut_tup[0]
            if clz in clz_ids_dict:
                clz_ids_dict[clz].append(mut_id)
            else:
                clz_ids_dict[clz] = [mut_id]
        for clz, ids in clz_ids_dict.items():
            total_runtime = 0
            for mut_id in ids:
                total_runtime += np.mean(id_runtimes_dict[mut_id])
            clz_runtimes_dict[clz].append(math.ceil(total_runtime))
    info_list = []
    for clz, runtimes in clz_runtimes_dict.items():
        avg_dev = round(np.mean([runtimes[0]/runtimes[i] for i in range(1, len(runtimes))]), 2)
        info_list.append([clz] + runtimes + [avg_dev])
    info_list.sort(key=lambda x: x[6], reverse=True)
    for info in info_list:
        df.loc[len(df.index)] = info
    df.to_csv(f'{analyzed_dir}/total_runtime/each_clazz/{p}.csv', sep=',', header=True, index=False)

In [3]:
import json
for p in project_list:
    with open(f'parsed_data/default_version/{p}/mutantId_mutantTuple.json', 'r') as f:
        id_tuple_dict = json.load(f)
    clazz_set = set()
    for i, t in id_tuple_dict.items():
        clazz_set.add(t[0])
    print(p, len(clazz_set))

assertj-assertions-generator 13
commons-cli 28
commons-csv 10
commons-codec 75
delight-nashorn-sandbox 18
empire-db 64
jimfs 102
handlebars.java 165
httpcore 219
riptide 44
commons-net 86
commons-collections 437
guava 127
java-design-patterns 4
jooby 76
maven-dependency-plugin 49
maven-shade-plugin 32
sling-org-apache-sling-auth-core 16
stream-lib 44


In [ ]:
import pandas as pd
import numpy as np
import json

# m1,m2,m3,m4....
# t1, t2
# t1 only covers lines in the mutated class
# t2 never covers any line in the mutated class
# if we run t1 on each mutant, would it run slow every single mutant? because the code it covers always has to be re-optimized due to hotswapping
# meanwhile, if we run t2 on each mutant, would it run faster, since it doesn't depend on the mutated class that gets hotswapped
# imagine large class
# many, many lines -> many, many mutation possibilities
# have one test that calls every method in the class
# only covers lines in that class
# another test that calls just one method or covers only one line in that class
# but it calls methods in every other class there

# mutated_clazz = 'org.apache.commons.math4.legacy.core.dfp.DfpMath'
project_name = 'commons-math'
junit_version = 'junit4'
clazz_index = 0
path = f'{project_name}/result'
with open(f'{path}/mutantId_mutantTuple.json', 'r') as file:
    id_tuple_dict = json.load(file)
with open(f'{path}/mutantId_runtimeList.json', 'r') as file:
    id_runtimes_dict = json.load(file)
with open(f'{path}/mutantId_testsInOrder.json', 'r') as file:
    id_tests_dict = json.load(file)

test_clazzes_dict = dict()
for mut_id, mut_tup in id_tuple_dict.items():
    tests = id_tests_dict[mut_id]
    clazz = mut_tup[clazz_index]
    for t in tests:
        if t in test_clazzes_dict:
            test_clazzes_dict[t].append(clazz)
        else:
            test_clazzes_dict[t] = [clazz]

test_clazzes_dict = {k: list(set(v)) for k, v in test_clazzes_dict.items()}
print(test_clazzes_dict)

{'org.apache.commons.math4.legacy.core.dfp.DfpMathTest.testSin(org.apache.commons.math4.legacy.core.dfp.DfpMathTest)': ['org.apache.commons.math4.legacy.core.dfp.DfpMath'], 'org.apache.commons.math4.legacy.core.dfp.DfpMathTest.testPow(org.apache.commons.math4.legacy.core.dfp.DfpMathTest)': ['org.apache.commons.math4.legacy.core.dfp.DfpMath']}


## 1 Get total running time

In [2]:
import pandas as pd
import numpy as np
import json
from scipy.stats import ttest_ind, mannwhitneyu
choice = 'more_projects'


def get_info(file_path):
    with open(f'{file_path}/mutantId_runtimeList.json', 'r') as f:
        id_runtimes_dict = json.load(f)
    return id_runtimes_dict


significant_df = pd.DataFrame(None, columns=['project', 'seed1', 'seed2', 'T-test', 'U-test'])
for project in project_list:
    columns = ['seed'] + [f'round{i}' for i in range(round_number)]
    columns += ['avg.', '/avg. default', '/avg. fastest']
    runtime_df = pd.DataFrame(None, columns=columns)
    seed_runtimes_dict = {}
    for seed in seed_list:
        runtime_list = np.zeros(round_number, dtype=int)
        with open(f'{analyzed_dir}/mutant_list/non-flaky/{project}_{seed}.json', 'r') as f:
            non_flaky_id_list = json.load(f)
        per_id_runtimes_dict = get_info(f'{parsed_dir}/{project}_{seed}')
        for mut_id in non_flaky_id_list:
            runtime_list += np.array(per_id_runtimes_dict[mut_id])
        seed_runtimes_dict[seed] = runtime_list

    avg_default = int(round(np.mean(seed_runtimes_dict['default'])))
    # avg_fastest = int(round(np.mean(seed_runtimes_dict['fastest'])))
    for i in range(seed_number):
        seed1 = seed_list[i]
        runtimes1 = seed_runtimes_dict[seed1]
        avg_runtime = int(round(np.mean(runtimes1)))
        ratio_vs_default = round(avg_runtime / avg_default, 3)
        # ratio_vs_fastest = round(avg_runtime / avg_fastest, 3)
        infos = [seed1] + list(runtimes1) + [f'{avg_runtime}', f'{ratio_vs_default}', f'']
        runtime_df.loc[len(runtime_df.index)] = infos
        for j in range(seed_number):
            if j > i:
                seed2 = seed_list[j]
                runtimes2 = seed_runtimes_dict[seed2]
                t_stat, t_p_value = ttest_ind(runtimes1, runtimes2)
                u_stat, u_p_value = mannwhitneyu(runtimes1, runtimes2)
                significant_df.loc[len(significant_df.index)] = [project, seed1, seed2, f'{t_p_value:.3f}', f'{u_p_value:.3f}']
    runtime_df.to_csv(f'{analyzed_dir}/total_runtime/{project}.csv', sep=',', header=True, index=False)
significant_df.to_csv(f'{analyzed_dir}/total_runtime/significant_results.csv', sep=',', header=True, index=False)

## 2 Calculate T-test, U-test

In [ ]:
import pandas as pd
import warnings
import json
from scipy.stats import ttest_ind, mannwhitneyu
from pitest_log_parser import mutant_choice, test_choice, TIMED_OUT
random_mutant = False
random_test = True
choice = f'{mutant_choice[random_mutant]}_{test_choice[random_test]}'
warnings.filterwarnings('ignore', category=RuntimeWarning, message='Precision loss occurred in moment calculation.')


def get_info(project, seed):
    path = f'{parsed_path}/{choice}/{project}_{seed}'
    with open(f'{path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{path}/mutantId_runtimeList.json', 'r') as file:
        mutantId_runtimeList_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_runtimeList_dict


def to_be_calculable(runtime_list):
    output_list = []
    for runtime in runtime_list:
        if runtime == TIMED_OUT:
            output_list.append(6000)
        else:
            output_list.append(runtime)
    return output_list


for project in project_list:
    seed_mutants_dict = {}
    for seed in seed_list:
        mutantId_mutantTuple_dict, mutantId_runtimeList_dict = get_info(project, seed)
        with open(f'{analyzed_path}/{choice}/mutant_list/non-flaky/{project}_{seed}.json', 'r') as file:
            non_flaky_list = json.load(file)
        seed_mutants_dict[seed] = {}
        for mutant_id in non_flaky_list:
            mutant_pref = tuple(mutantId_mutantTuple_dict[mutant_id][:5])
            seed_mutants_dict[seed][mutant_pref] = to_be_calculable(mutantId_runtimeList_dict[mutant_id])
    for i in range(seed_number):
        i_seed = seed_list[i]
        i_mutants_dict = seed_mutants_dict[i_seed]
        j = i + 1
        while j < seed_number:
            df = pd.DataFrame(None, columns=['mutant', 'T-test', 'U-test'])
            j_seed = seed_list[j]
            j_mutants_dict = seed_mutants_dict[j_seed]
            significant_number = 0
            for pref, i_runtime_list in i_mutants_dict.items():
                j_runtime_list = j_mutants_dict[pref]
                t_stat, t_p_value = ttest_ind(i_runtime_list, j_runtime_list)
                u_stat, u_p_value = mannwhitneyu(i_runtime_list, j_runtime_list)
                if t_p_value < 0.05 or u_p_value < 0.05:
                    significant_number += 1
                df.loc[len(df.index)] = [pref, t_p_value, u_p_value]
            # df.to_csv(f'{analyzed_path}/{choice}/significance_detection/non-flaky/{project}_{i_seed}_{j_seed}.csv', sep=',', header=True, index=False)
            j += 1
            print(f'{project} with ({i_seed}, {j_seed}): {significant_number}/{len(non_flaky_list)}')

## 3. Calculate proportions

In [ ]:
import pandas as pd
import random
import json
from pitest_log_parser import mutant_choice, test_choice, TIMED_OUT
random_mutant = False
random_test = True
choice = f'{mutant_choice[random_mutant]}_{test_choice[random_test]}'

def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as file:
        mutantId_testsInOrder_dict = json.load(file)
    with open(f'{file_path}/mutantId_runtimeList.json', 'r') as file:
        mutantId_runtimeList_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_testsInOrder_dict, mutantId_runtimeList_dict


for project in project_list:
    total_flaky_tests = set()
    for seed in seed_list:
        flaky_chaotic_order_mutant_list = []
        total_num = 0
        non_flaky_num = 0
        flaky_chaotic_order_num = 0
        uniqueId_mutantIds_dict = {}
        mutantId_mutantTuple_dict, mutantId_testsInOrder_dict, mutantId_runtimeList_dict = get_info(f'parsed_data/{choice}/{project}_{seed}')
        for mutant_id, mutant_tuple in mutantId_mutantTuple_dict.items():
            unique_id = tuple(mutant_tuple[:5])
            test_list = mutantId_testsInOrder_dict[mutant_id]
            if unique_id not in uniqueId_mutantIds_dict:
                uniqueId_mutantIds_dict[unique_id] = set()
            uniqueId_mutantIds_dict[unique_id].add(mutant_id)
        total_num = len(uniqueId_mutantIds_dict)
        for unique_id, mutant_ids in uniqueId_mutantIds_dict.items():
            if len(mutant_ids) == 1:
                non_flaky_num += 1
            else:
                test_intersection_set = None
                for mutant_id in mutant_ids:
                    per_test_set = set(mutantId_testsInOrder_dict[mutant_id])
                    if test_intersection_set is None:
                        test_intersection_set = per_test_set
                    else:
                        test_intersection_set &= per_test_set
                is_same = True
                flaky_tests = set()
                for mutant_id in mutant_ids:
                    per_test_set = set(mutantId_testsInOrder_dict[mutant_id])
                    if len(per_test_set) == len(test_intersection_set):
                        continue
                    flaky_tests |= (per_test_set - test_intersection_set)
                if len(flaky_tests) == 0:
                    flaky_chaotic_order_num += 1
                    flaky_chaotic_order_mutant_list.append(random.choice(list(mutant_ids)))
                total_flaky_tests |= flaky_tests
        # with open(f'analyzed_data/{choice}/mutant_list/chaotic_order/{project}_{seed}.json', 'w') as file:
        #     json.dump(flaky_chaotic_order_mutant_list, file, indent=4)
    
        print(f'{project} with {seed}:')
        print(f'Total mutants: {total_num}')
        print(f'Non-flaky mutants: {non_flaky_num} ({non_flaky_num / total_num:.4f})')
        print(f'Flaky mutants due to chaotic test list: {flaky_chaotic_order_num} ({flaky_chaotic_order_num / total_num:.4f})')

## 4. Create guiding files

In [ ]:
import random
import json
from pitest_log_parser import project_junitVersion_dict, mutant_choice, test_choice, TIMED_OUT
random_mutant = False
random_test = True
choice = f'{mutant_choice[random_mutant]}_{test_choice[random_test]}'

def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as file:
        mutantId_testsInOrder_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_testsInOrder_dict


def mutant_to_json(mutant, test_list, junit_version):
    def test_sort(test):
        return (test['name'])
    
    test_order = []
    if junit_version == 'junit4':
        for test in test_list:
            paren_index = test.find('(')
            for i in range(paren_index, 0, -1):
                if test[i] == '.': break
            test_order.append({
                'definingClass': test[:i],
                'name': test
            })
    else:
        for test in test_list:
            class_end_index = test.find('[')
            test_order.append({
                'definingClass': test[:class_end_index-1],
                'name': test
            })
    if random_test:
        random.shuffle(test_order)
    else:
        test_order.sort(key=test_sort)

    return {
        'id': {
            'location': {
                'clazz': mutant[0],
                'method': mutant[1],
                'methodDesc': mutant[2]
            },
            'indexes': f'[{mutant[3]}]',
            'mutator': mutant[4]
        },
        'filename': mutant[5],
        'block': f'[{mutant[6]}]',
        'lineNumber': int(mutant[7]),
        'description': mutant[8],
        'testsInOrder': test_order
    }


def mutant_sort(mutant):
    return (
        mutant['id']['location']['clazz'],
        mutant['id']['location']['method'],
        mutant['id']['location']['methodDesc'],
        mutant['id']['indexes'],
        mutant['id']['mutator']
    )


parent_path = f'analyzed_data/{choice}/mutant_list'
for project in project_list:
    junit_version = project_junitVersion_dict[project]
    # get the mutant intersection
    id_set = set()
    for seed in seed_list:
        mutantId_mutantTuple_dict, _ = get_info(f'parsed_data/{choice}/{project}_{seed}')
        with open(f'{parent_path}/chaotic_order/{project}_{seed}.json', 'r') as file:
            chaotic_order_list = json.load(file)
        per_id_set = set()
        for mutant_id in chaotic_order_list:
            per_id_set.add(tuple(mutantId_mutantTuple_dict[mutant_id][:5]))
        if len(id_set) == 0:
            id_set |= per_id_set
        else:
            id_set &= per_id_set
    # create guiding mutant list
    for seed in seed_list:
        with open(f'{parent_path}/non-flaky/{project}_{seed}.json', 'r') as file:
            non_flaky_list = json.load(file)
        mutantId_mutantTuple_dict, mutantId_testsInOrder_dict = get_info(f'parsed_data/{choice}/{project}_{seed}')
        output_list = []
        id_visited_dict = {item: True for item in id_set}
        for mutant_id, mutant_tuple in mutantId_mutantTuple_dict.items():
            id_tuple = tuple(mutant_tuple[:5])
            if mutant_id in non_flaky_list or (id_tuple in id_set and id_visited_dict[id_tuple]):
                if id_tuple in id_set:
                    id_visited_dict[id_tuple] = False
                output_list.append(mutant_to_json(mutant=mutant_tuple,
                                                  test_list=mutantId_testsInOrder_dict[mutant_id],
                                                  junit_version=junit_version))
        if random_mutant:
            random.shuffle(output_list)
        else:
            output_list.sort(key=mutant_sort)
        with open(f'analyzed_data/{choice}/guiding_files/{project}_{seed}.json', 'w') as f:
            f.write(json.dumps(output_list, indent=4))

## 5. Choose fastest test order to guide

In [ ]:
import json
import numpy as np
from pitest_log_parser import project_junitVersion_dict
seeds = [0, 42, 123, 216, 1202, 1999, 2002, 2024, 31415, 99999]
choice = 'more_projects'

def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{file_path}/mutantId_runtimeList.json', 'r') as file:
        mutantId_runtimeList_dict = json.load(file)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as file:
        mutantId_testsInOrder_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_runtimeList_dict, mutantId_testsInOrder_dict
    

def mutant_sort(mutant):
    return (
        mutant['id']['location']['clazz'],
        mutant['id']['location']['method'],
        mutant['id']['location']['methodDesc'],
        mutant['id']['indexes'],
        mutant['id']['mutator']
    )
    

def mutant_to_json(mutant, test_list, junit_version):
    test_order = []
    if junit_version == 'junit4':
        for test in test_list:
            paren_index = test.find('(')
            for i in range(paren_index, 0, -1):
                if test[i] == '.': break
            test_order.append({
                'definingClass': test[:i],
                'name': test
            })
    else:
        for test in test_list:
            class_end_index = test.find('[')
            test_order.append({
                'definingClass': test[:class_end_index-1],
                'name': test
            })
    return {
        'id': {
            'location': {
                'clazz': mutant[0],
                'method': mutant[1],
                'methodDesc': mutant[2]
            },
            'indexes': f'[{mutant[3]}]',
            'mutator': mutant[4]
        },
        'testsInOrder': test_order
    }


for project in ['commons-cli']:
    junit_version = project_junitVersion_dict[project]
    mutation_runtimes_dict = {}
    mutation_tests_dict = {}
    is_recorded = False
    for seed in seeds:
        with open(f'{analyzed_dir}/{choice}/mutant_list/non-flaky/{project}_{seed}.json', 'r') as f:
            non_flaky_list = json.load(f)
        id_tuple_dict, id_runtimes_dict, id_tests_dict = get_info(f'{parsed_dir}/{choice}/{project}_{seed}')
        for mut_id in non_flaky_list:
            mut = tuple(id_tuple_dict[mut_id])
            runtime_list = id_runtimes_dict[mut_id]
            runtime = round(np.mean(runtime_list))
            if is_recorded:
                mutation_runtimes_dict[mut].append(runtime)
                mutation_tests_dict[mut].append(id_tests_dict[mut_id])
            else:
                mutation_runtimes_dict[mut] = [runtime]
                mutation_tests_dict[mut] = [id_tests_dict[mut_id]]
        is_recorded = True
    output_list = []
    for mut, runtimes in mutation_runtimes_dict.items():
        tests = mutation_tests_dict[mut]
        mini = 1000000
        mini_index = -1
        for i in range(len(seeds)):
            if runtimes[i] < mini:
                mini = runtimes[i]
                mini_index = i
        output_list.append(mutant_to_json(mutant=mut,
                                          test_list=tests[mini_index],
                                          junit_version=junit_version))
    output_list.sort(key=mutant_sort)
    with open(f'{analyzed_dir}/{choice}/guiding_files/{project}_fastest.json', 'w') as f:
        f.write(json.dumps(output_list, indent=4))

## 6. Generate input files based on single default result

In [ ]:
import random
import json
from pitest_log_parser import project_junitVersion_dict


def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as file:
        mutantId_testsInOrder_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_testsInOrder_dict


def mutant_to_json(mutant, test_list, junit_version):
    test_order = []
    if junit_version == 'junit4':
        for test in test_list:
            paren_index = test.find('(')
            for i in range(paren_index, 0, -1):
                if test[i] == '.': break
            test_order.append({
                'definingClass': test[:i],
                'name': test
            })
    else:
        for test in test_list:
            class_end_index = test.find('[')
            test_order.append({
                'definingClass': test[:class_end_index-1],
                'name': test
            })
    # random.shuffle(test_order)
    return {
        'id': {
            'location': {
                'clazz': mutant[0],
                'method': mutant[1],
                'methodDesc': mutant[2]
            },
            'indexes': f'[{mutant[3]}]',
            'mutator': mutant[4]
        },
        'filename': mutant[5],
        'block': f'[{mutant[6]}]',
        'lineNumber': int(mutant[7]),
        'description': mutant[8],
        'testsInOrder': test_order
    }


def mutant_sort(mutant):
    return (
        mutant['id']['location']['clazz'],
        mutant['id']['location']['method'],
        mutant['id']['location']['methodDesc'],
        mutant['id']['indexes'],
        mutant['id']['mutator']
    )


for project in project_list:
    junit_version = project_junitVersion_dict[project]
    id_tuple_dict, id_tests_dict = get_info(f'parsed_data/default_version/{project}')
    for seed in seed_list:
        # random.seed(seed)
        seed = 'default'
        output_list = []
        for mut_id, mut_tup in id_tuple_dict.items():
            output_list.append(mutant_to_json(mutant=mut_tup,
                                              test_list=id_tests_dict[mut_id],
                                              junit_version=junit_version))
        output_list.sort(key=mutant_sort)
        with open(f'analyzed_data/default_version/guiding_files/{project}_{seed}.json', 'w') as f:
            f.write(json.dumps(output_list, indent=4))

## 7. Check the order of mutants

In [ ]:
import json
from flakiness_filter import clazz_index
choice = 'more_projects'


print('For each clazz (each process):')
for project in project_list:
    clazz_orders_dict = dict()
    for seed in seed_list:
        cur_clazz_orders_dict = dict()
        with open(f'{parsed_dir}/{choice}/{project}_{seed}/mutantId_mutantTuple.json', 'r') as f:
            id_tuple_dict = json.load(f)
        with open(f'{analyzed_dir}/{choice}/mutant_list/non-flaky/{project}_{seed}.json', 'r') as f:
            non_flaky_id_list = json.load(f)

        for mut_id in non_flaky_id_list:
                mut_tup = id_tuple_dict[mut_id]
                clazz = mut_tup[clazz_index]
                if clazz in cur_clazz_orders_dict:
                    cur_clazz_orders_dict[clazz].append(mut_tup)
                else:
                    cur_clazz_orders_dict[clazz] = [mut_tup]
        diff_cnt = 0
        if len(clazz_orders_dict) == 0:
            clazz_orders_dict = cur_clazz_orders_dict
        else:
            for clazz, mutant_order in clazz_orders_dict.items():
                if mutant_order != cur_clazz_orders_dict[clazz]:
                    diff_cnt += 1
        if diff_cnt > 0:
            print(f'The diff cnt of {project} with {seed}: {diff_cnt}')

## 8. Average runtime (per mutant) table

In [ ]:
import pandas as pd
import numpy as np


def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{file_path}/mutantId_runtimeList.json', 'r') as file:
        mutantId_runtimeList_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_runtimeList_dict


choice = 'more_projects'
cols = ['clazz', 'method', 'methodDesc', 'indexes', 'mutator'] + [f'{s}' for s in seed_list]
for project in project_list:
    df = pd.DataFrame(None, columns=cols)
    mutant_runtimes_dict = dict()
    is_recorded = False
    for seed in seed_list:
        id_tuple_dict, id_runtimes_dict = get_info(f'{parsed_path}/{choice}/{project}_{seed}')
        with open(f'{analyzed_path}/{choice}/mutant_list/non-flaky/{project}_{seed}.json', 'r') as f:
            non_flaky_id_list = json.load(f)
        for mut_id in non_flaky_id_list:
            mut_tup = tuple(id_tuple_dict[mut_id])
            avg_runtime = round(np.mean(id_runtimes_dict[mut_id]))
            if is_recorded:
                mutant_runtimes_dict[mut_tup].append(avg_runtime)
            else:
                mutant_runtimes_dict[mut_tup] = [avg_runtime]
        is_recorded = True
    for mut, avg_runtimes in mutant_runtimes_dict.items():
        df.loc[len(df.index)] = list(mut) + avg_runtimes
    df.to_csv(f'{analyzed_path}/{choice}/runtime_per_mutant/{project}.csv', sep=',', header=True, index=False)

## 9. Based on test scores
score = w_kill * kill_ratio + w_cover * cover_ratio \
w_kill + w_cover = 1

For kill_ratio: \
A test covers n mutants, and it can kill m(0≤m≤n) mutants, then kill_ratio = m / n; \
For cover_ratio: \
There are n mutants from a class, and a test can cover m mutants(0≤m≤n), then cover_ratio = m / n;

The higher the score, the better the test.

In [ ]:
import json
import math
import numpy as np
from pitest_log_parser import project_junitVersion_dict
choice = 'more_projects'
clazz_index = 0


def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as file:
        mutantId_testsInOrder_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_testsInOrder_dict
    

def mutant_sort(mutant):
    return (
        mutant['id']['location']['clazz'],
        mutant['id']['location']['method'],
        mutant['id']['location']['methodDesc'],
        mutant['id']['indexes'],
        mutant['id']['mutator']
    )
    

def mutant_to_json(mutant, test_list, junit_version):
    test_order = []
    if junit_version == 'junit4':
        for test in test_list:
            paren_index = test.find('(')
            for i in range(paren_index, 0, -1):
                if test[i] == '.': break
            test_order.append({
                'definingClass': test[:i],
                'name': test
            })
    else:
        for test in test_list:
            class_end_index = test.find('[')
            test_order.append({
                'definingClass': test[:class_end_index-1],
                'name': test
            })
    return {
        'id': {
            'location': {
                'clazz': mutant[0],
                'method': mutant[1],
                'methodDesc': mutant[2]
            },
            'indexes': f'[{mutant[3]}]',
            'mutator': mutant[4]
        },
        'testsInOrder': test_order
    }


for project in project_list:
    junit_version = project_junitVersion_dict[project]
    with open(f'{analyzed_dir}/{choice}/mutant_list/non-flaky/{project}_default.json', 'r') as f:
        non_flaky_id_list = json.load(f)
    with open(f'{analyzed_dir}/{choice}/test_scores/{project}.json', 'r') as f:
        test_score_dict = {eval(k): v for k, v in json.load(f).items()}
    id_tuple_dict, id_tests_dict = get_info(f'{parsed_dir}/{choice}/{project}_default')
    output_list = []
    for mut_id in non_flaky_id_list:
        index_score_tup_list = []
        mut_tup = id_tuple_dict[mut_id]
        clazz = mut_tup[clazz_index]
        tests = id_tests_dict[mut_id]
        for i, t in enumerate(tests):
            index_score_tup_list.append((i, test_score_dict[(clazz, t)]))
        index_score_tup_list.sort(key=lambda x: x[1], reverse=True)
        test_list = []
        num = math.ceil(0.5 * (i + 1))
        for i in range(num):
            test_list.append(tests[index_score_tup_list[i][0]])
        output_list.append(mutant_to_json(mutant=mut_tup,
                                          test_list=test_list,
                                          junit_version=junit_version))
        output_list.sort(key=mutant_sort)
        with open(f'{analyzed_dir}/{choice}/guiding_files/{project}_half.json', 'w') as f:
            f.write(json.dumps(output_list, indent=4))

In [ ]:
# What's the result?
import json
import numpy as np
from pitest_log_parser import project_junitVersion_dict
choice = 'more_projects'
clazz_index = 0
method_index = 1
methodDesc_index = 2
indexes_index = 3
mutator_index = 4
STATUS = 'status'
KILLED = 'KILLED'
SURVIVED = 'SURVIVED'


def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{file_path}/mutantId_runtimeList.json', 'r') as file:
        mutantId_runtimeList_dict = json.load(file)
    with open(f'{file_path}/mutations_xml.json', 'r') as file:
        mutants = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_runtimeList_dict, mutants


def xml_to_key(obj):
    return (obj['mutatedClass'],
            obj['mutatedMethod'],
            obj['methodDescription'],
            str([int(i) for i in obj['indexes']]),
            obj['mutator'])


def log_to_key(obj):
    return (obj[clazz_index],
            obj[method_index],
            obj[methodDesc_index],
            str([int(i) for i in obj[indexes_index].split(', ')]),
            obj[mutator_index])


def get_deleted_result(file_path, idx, title):
    id_tuple_dict, id_runtimes_dict, xml_infos_list = get_info(file_path)
    non_flaky_mutants = set()
    for mut_id, runtimes in id_runtimes_dict.items():
        if np.isnan(runtimes).any():
            continue
        non_flaky_mutants.add(log_to_key(id_tuple_dict[mut_id]))
    killed_num = 0
    for mut in xml_infos_list:
        if xml_to_key(mut) in non_flaky_mutants:
            if mut[STATUS] == KILLED:
                killed_num += 1
    print(f'{idx}. {title} test(s) with the highest score: {killed_num} / {len(non_flaky_mutants)} = {killed_num / len(non_flaky_mutants):.3f}')


for project in project_list:
    mutants = set()
    is_recorded = False
    print(f'For {project}:')
    for i, seed in enumerate(seed_list):
        with open(f'{analyzed_dir}/{choice}/mutant_list/non-flaky/{project}_{seed}.json', 'r') as f:
            non_flaky_id_list = json.load(f)
        id_tuple_dict, id_runtimes_dict, xml_infos_list = get_info(f'{parsed_dir}/{choice}/{project}_{seed}')
        if not is_recorded:
            for mut_id in non_flaky_id_list:
                mut_tup = id_tuple_dict[mut_id]
                mutants.add(log_to_key(mut_tup))
        killed_num = 0
        for mut in xml_infos_list:
            if xml_to_key(mut) in mutants:
                if mut[STATUS] == KILLED:
                    killed_num += 1
        print(f'{i + 1}. Seed is {seed}: {killed_num} / {len(mutants)} = {killed_num / len(mutants):.3f}')
    get_deleted_result(f'{parsed_dir}/{choice}/{project}_single', i + 2, 'Single')
    get_deleted_result(f'{parsed_dir}/{choice}/{project}_half', i + 3, 'Half')

## 10. Reorder mutants or tests

In [ ]:
# Keep mutant order.
import numpy as np
import copy
import json
from pitest_log_parser import project_junitVersion_dict
choice = 'more_projects'
clazz_index = 0
tests_index = 5

def mutant_sort(mutant):
    return mutant[:5]


def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as f:
        id_tuple_dict = json.load(f)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as f:
        id_tests_dict = json.load(f)
    return id_tuple_dict, id_tests_dict


def mutant_to_json(mutant, test_list, junit_version):
    test_order = []
    if junit_version == 'junit4':
        for test in test_list:
            paren_index = test.find('(')
            for i in range(paren_index, 0, -1):
                if test[i] == '.': break
            test_order.append({
                'definingClass': test[:i],
                'name': test
            })
    else:
        for test in test_list:
            class_end_index = test.find('[')
            test_order.append({
                'definingClass': test[:class_end_index - 1],
                'name': test
            })
    return {
        'id': {
            'location': {
                'clazz': mutant[0],
                'method': mutant[1],
                'methodDesc': mutant[2]
            },
            'indexes': f'[{mutant[3]}]',
            'mutator': mutant[4]
        },
        'testsInOrder': test_order
    }


for project in project_list:
    id_tuple_dict, id_tests_dict = get_info(f'parsed_data/default_version/{project}')
    reordered_mutants = list()
    for mut_id, mut_tup in id_tuple_dict.items():
        mut = mut_tup[:5] + [id_tests_dict[mut_id]]
        reordered_mutants.append(mut)
    reordered_mutants.sort(key=mutant_sort)
    output_list = list()
    recorded_clazz_set = set()
    stack = list()
    for mut in reordered_mutants:
        clazz = mut[clazz_index]
        tests = mut[tests_index]
        if clazz in recorded_clazz_set:
            reordered_tests = list()
            cur_stack = list()
            remains = copy.deepcopy(tests)
            while len(stack) > 0:
                t = stack.pop()
                if t in tests:
                    reordered_tests.append(t)
                    cur_stack.append(t)
                    remains.remove(t)
            reordered_tests += remains
            stack = cur_stack + remains
            output_list.append(mutant_to_json(mutant=mut,
                                              test_list=reordered_tests,
                                              junit_version=project_junitVersion_dict[project]))
        else:
            recorded_clazz_set.add(clazz)
            stack = list()
            for t in tests:
                stack.append(t)
            output_list.append(mutant_to_json(mutant=mut,
                                              test_list=tests,
                                              junit_version=project_junitVersion_dict[project]))
    with open(f'{analyzed_dir}/{choice}/guiding_files/{project}_GC_torder.json', 'w') as f:
        f.write(json.dumps(output_list, indent=4))

In [ ]:
# Keep test order: with the most tests
import copy
import json
from pitest_log_parser import project_junitVersion_dict
choice = 'more_projects'


def mutant_sort(mutant):
    return (
        mutant['id']['location']['clazz'],
        -len(mutant['testsInOrder'])
    )


def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as f:
        id_tuple_dict = json.load(f)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as f:
        id_tests_dict = json.load(f)
    return id_tuple_dict, id_tests_dict


def mutant_to_json(mutant, test_list, junit_version):
    test_order = []
    if junit_version == 'junit4':
        for test in test_list:
            paren_index = test.find('(')
            for i in range(paren_index, 0, -1):
                if test[i] == '.': break
            test_order.append({
                'definingClass': test[:i],
                'name': test
            })
    else:
        for test in test_list:
            class_end_index = test.find('[')
            test_order.append({
                'definingClass': test[:class_end_index - 1],
                'name': test
            })
    return {
        'id': {
            'location': {
                'clazz': mutant[0],
                'method': mutant[1],
                'methodDesc': mutant[2]
            },
            'indexes': f'[{mutant[3]}]',
            'mutator': mutant[4]
        },
        'testsInOrder': test_order
    }


for project in project_list:
    id_tuple_dict, id_tests_dict = get_info(f'parsed_data/default_version/{project}')
    output_list = list()
    for mut_id, mut_tup in id_tuple_dict.items():
        output_list.append(mutant_to_json(mutant=mut_tup,
                                          test_list=id_tests_dict[mut_id],
                                          junit_version=project_junitVersion_dict[project]))
    output_list.sort(key=mutant_sort)
    with open(f'{analyzed_dir}/{choice}/guiding_files/{project}_GC_morder.json', 'w') as f:
        f.write(json.dumps(output_list, indent=4))

In [5]:
# Keep test order: with the fewest tests
import json
from pitest_log_parser import project_junitVersion_dict
choice = 'more_projects'


def mutant_sort(mutant):
    return (
        mutant['id']['location']['clazz'],
        len(mutant['testsInOrder'])
    )


def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as f:
        id_tuple_dict = json.load(f)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as f:
        id_tests_dict = json.load(f)
    return id_tuple_dict, id_tests_dict


def mutant_to_json(mutant, test_list, junit_version):
    test_order = []
    if junit_version == 'junit4':
        for test in test_list:
            paren_index = test.find('(')
            for i in range(paren_index, 0, -1):
                if test[i] == '.': break
            test_order.append({
                'definingClass': test[:i],
                'name': test
            })
    else:
        for test in test_list:
            class_end_index = test.find('[')
            test_order.append({
                'definingClass': test[:class_end_index - 1],
                'name': test
            })
    return {
        'id': {
            'location': {
                'clazz': mutant[0],
                'method': mutant[1],
                'methodDesc': mutant[2]
            },
            'indexes': f'[{mutant[3]}]',
            'mutator': mutant[4]
        },
        'testsInOrder': test_order
    }


for project in project_list:
    id_tuple_dict, id_tests_dict = get_info(f'parsed_data/default_version/{project}')
    output_list = list()
    for mut_id, mut_tup in id_tuple_dict.items():
        output_list.append(mutant_to_json(mutant=mut_tup,
                                          test_list=id_tests_dict[mut_id],
                                          junit_version=project_junitVersion_dict[project]))
    output_list.sort(key=mutant_sort)
    with open(f'{analyzed_dir}/{choice}/guiding_files/{project}_M_fewest_tests.json', 'w') as f:
        f.write(json.dumps(output_list, indent=4))

In [17]:
# Keep test order: with the most coverage
import json
from pitest_log_parser import project_junitVersion_dict
choice = 'more_projects'
clazz_index = 0
lineNum_index = 7


def mutant_sort(mutant):
    return (
        mutant['id']['location']['clazz'],
        -mutant['metric']
    )

def mutant_sort(mutant):
    return (
        mutant['id']['location']['clazz'],
        -mutant['metric']
    )


def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as f:
        id_tuple_dict = json.load(f)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as f:
        id_tests_dict = json.load(f)
    return id_tuple_dict, id_tests_dict


def mutant_to_json(mutant, test_list, junit_version):
    test_order = []
    mut_set = set()
    if junit_version == 'junit4':
        for test in test_list:
            mut_set |= set(test_ids_dict[test])
            paren_index = test.find('(')
            for i in range(paren_index, 0, -1):
                if test[i] == '.': break
            test_order.append({
                'definingClass': test[:i],
                'name': test
            })
    else:
        for test in test_list:
            mut_set |= set(test_ids_dict[test])
            class_end_index = test.find('[')
            test_order.append({
                'definingClass': test[:class_end_index - 1],
                'name': test
            })
    return {
        'id': {
            'location': {
                'clazz': mutant[0],
                'method': mutant[1],
                'methodDesc': mutant[2]
            },
            'indexes': f'[{mutant[3]}]',
            'mutator': mutant[4]
        },
        'testsInOrder': test_order,
        'metric': len(mut_set)
    }


for project in project_list:
    id_tuple_dict, id_tests_dict = get_info(f'parsed_data/default_version/{project}')
    output_list = list()
    test_ids_dict = dict()
    test_clazzes_dict = dict()
    for mut_id, mut_tup in id_tuple_dict.items():
        tests = id_tests_dict[mut_id]
        clazz = mut_tup[clazz_index]
        for t in tests:
            if t in test_ids_dict:
                test_clazzes_dict[t].append(clazz)
                test_ids_dict[t].append(mut_id)
            else:
                test_clazzes_dict[t] = [clazz]
                test_ids_dict[t] = [mut_id]
    
    for mut_id, mut_tup in id_tuple_dict.items():
        output_list.append(mutant_to_json(mutant=mut_tup,
                                          test_list=id_tests_dict[mut_id],
                                          junit_version=project_junitVersion_dict[project]))
    output_list.sort(key=mutant_sort)
    # with open(f'{analyzed_dir}/{choice}/guiding_files/{project}_M_most_coverage.json', 'w') as f:
    #     f.write(json.dumps(output_list, indent=4))

    tmp = list()
    for t, clazzes in test_clazzes_dict.items():
        tmp.append([t, len(set(clazzes)), len(set(test_ids_dict[t]))])
    tmp.sort(key=lambda x: x[0])
    with open(f'controlled_analyzed_data/{choice}/extra/{project}_linesVSclazzes.json', 'w') as f:
        f.write(json.dumps(tmp, indent=4))

In [ ]:
# Keep test order: with the most similar
import json
from pitest_log_parser import project_junitVersion_dict
choice = 'more_projects'


def mutant_sort(mutant):
    return (
        mutant['id']['location']['clazz']
    )


def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as f:
        id_tuple_dict = json.load(f)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as f:
        id_tests_dict = json.load(f)
    return id_tuple_dict, id_tests_dict


def mutant_to_json(mutant, test_list, junit_version):
    test_order = []
    mut_set = set()
    if junit_version == 'junit4':
        for test in test_list:
            mut_set |= set(test_ids_dict[test])
            paren_index = test.find('(')
            for i in range(paren_index, 0, -1):
                if test[i] == '.': break
            test_order.append({
                'definingClass': test[:i],
                'name': test
            })
    else:
        for test in test_list:
            mut_set |= set(test_ids_dict[test])
            class_end_index = test.find('[')
            test_order.append({
                'definingClass': test[:class_end_index - 1],
                'name': test
            })
    return {
        'id': {
            'location': {
                'clazz': mutant[0],
                'method': mutant[1],
                'methodDesc': mutant[2]
            },
            'indexes': f'[{mutant[3]}]',
            'mutator': mutant[4]
        },
        'testsInOrder': test_order,
        'metric': tuple(mut_set)
    }


for project in project_list:
    id_tuple_dict, id_tests_dict = get_info(f'parsed_data/default_version/{project}')
    test_ids_dict = dict()
    for mut_id, mut_tup in id_tuple_dict.items():
        tests = id_tests_dict[mut_id]
        for t in tests:
            if t in test_ids_dict:
                test_ids_dict[t].append(mut_id)
            else:
                test_ids_dict[t] = [mut_id]
    mutant_list = list()
    for mut_id, mut_tup in id_tuple_dict.items():
        mutant_list.append(mutant_to_json(mutant=mut_tup,
                                          test_list=id_tests_dict[mut_id],
                                          junit_version=project_junitVersion_dict[project]))
    mutant_list.sort(key=mutant_sort)
    resorted_list = list()
    mutant_num = len(mutant_list)
    cur_clazz = ''
    is_next = False
    i = 0
    while i < mutant_num:
        mut = mutant_list[i]
        cur_set = set()
        if not is_next:
            is_next = True
            cur_clazz = mut['id']['location']['clazz']
            resorted_list.append(mut)
            cur_set = mut['metric']
            i += 1
        cur_mutants = list()
        while is_next and i < mutant_num:
            mut = mutant_list[i]
            if cur_clazz != mut['id']['location']['clazz']:
                is_next = False
                break
            cur_mutants.append(mut)
            i += 1
        while len(cur_mutants) > 0:
            best = 0
            best_idx = -1
            for j, mut in enumerate(cur_mutants):
                cnt = len(set(mut['metric']) & set(cur_set))
                if cnt > best:
                    best = cnt
                    best_idx = j
            resorted_list.append(cur_mutants[best_idx])
            cur_mutants.pop(best_idx)
            cur_set = resorted_list[-1]['metric']
    output_list = list()
    for mut in resorted_list:
        output_list.append({
        'id': {
            'location': {
                'clazz': mut['id']['location']['clazz'],
                'method': mut['id']['location']['method'],
                'methodDesc': mut['id']['location']['methodDesc']
            },
            'indexes': mut['id']['indexes'],
            'mutator': mut['id']['mutator']
        },
        'testsInOrder': mut['testsInOrder']
    })  
    with open(f'{analyzed_dir}/{choice}/guiding_files/{project}_M_most_similar.json', 'w') as f:
        f.write(json.dumps(output_list, indent=4))

In [10]:
# Keep test order: with the most different
import json
from pitest_log_parser import project_junitVersion_dict
choice = 'more_projects'


def mutant_sort(mutant):
    return (
        mutant['id']['location']['clazz']
    )


def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as f:
        id_tuple_dict = json.load(f)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as f:
        id_tests_dict = json.load(f)
    return id_tuple_dict, id_tests_dict


def mutant_to_json(mutant, test_list, junit_version):
    test_order = []
    mut_set = set()
    if junit_version == 'junit4':
        for test in test_list:
            mut_set |= set(test_ids_dict[test])
            paren_index = test.find('(')
            for i in range(paren_index, 0, -1):
                if test[i] == '.': break
            test_order.append({
                'definingClass': test[:i],
                'name': test
            })
    else:
        for test in test_list:
            mut_set |= set(test_ids_dict[test])
            class_end_index = test.find('[')
            test_order.append({
                'definingClass': test[:class_end_index - 1],
                'name': test
            })
    return {
        'id': {
            'location': {
                'clazz': mutant[0],
                'method': mutant[1],
                'methodDesc': mutant[2]
            },
            'indexes': f'[{mutant[3]}]',
            'mutator': mutant[4]
        },
        'testsInOrder': test_order,
        'metric': tuple(mut_set)
    }


for project in project_list:
    id_tuple_dict, id_tests_dict = get_info(f'parsed_data/default_version/{project}')
    test_ids_dict = dict()
    for mut_id, mut_tup in id_tuple_dict.items():
        tests = id_tests_dict[mut_id]
        for t in tests:
            if t in test_ids_dict:
                test_ids_dict[t].append(mut_id)
            else:
                test_ids_dict[t] = [mut_id]
    mutant_list = list()
    for mut_id, mut_tup in id_tuple_dict.items():
        mutant_list.append(mutant_to_json(mutant=mut_tup,
                                          test_list=id_tests_dict[mut_id],
                                          junit_version=project_junitVersion_dict[project]))
    mutant_list.sort(key=mutant_sort)
    resorted_list = list()
    mutant_num = len(mutant_list)
    cur_clazz = ''
    is_next = False
    i = 0
    while i < mutant_num:
        mut = mutant_list[i]
        cur_set = set()
        if not is_next:
            is_next = True
            cur_clazz = mut['id']['location']['clazz']
            resorted_list.append(mut)
            cur_set = mut['metric']
            i += 1
        cur_mutants = list()
        while is_next and i < mutant_num:
            mut = mutant_list[i]
            if cur_clazz != mut['id']['location']['clazz']:
                is_next = False
                break
            cur_mutants.append(mut)
            i += 1
        while len(cur_mutants) > 0:
            worse = 10000
            worse_idx = -1
            for j, mut in enumerate(cur_mutants):
                cnt = len(set(mut['metric']) & set(cur_set))
                if cnt < worse:
                    worse = cnt
                    worse_idx = j
            resorted_list.append(cur_mutants[worse_idx])
            cur_mutants.pop(worse_idx)
            cur_set = resorted_list[-1]['metric']
    output_list = list()
    for mut in resorted_list:
        output_list.append({
        'id': {
            'location': {
                'clazz': mut['id']['location']['clazz'],
                'method': mut['id']['location']['method'],
                'methodDesc': mut['id']['location']['methodDesc']
            },
            'indexes': mut['id']['indexes'],
            'mutator': mut['id']['mutator']
        },
        'testsInOrder': mut['testsInOrder']
    })  
    with open(f'{analyzed_dir}/{choice}/guiding_files/{project}_M_most_different.json', 'w') as f:
        f.write(json.dumps(output_list, indent=4))